In [1]:
!pip install git+https://github.com/CornellNLP/Cornell-Conversational-Analysis-Toolkit.git --user --upgrade

  Cloning https://github.com/CornellNLP/Cornell-Conversational-Analysis-Toolkit.git to c:\users\jonat\appdata\local\temp\pip-req-build-ffhip93l
  Resolved https://github.com/CornellNLP/Cornell-Conversational-Analysis-Toolkit.git to commit 3b0e4a14fdb12e28f29de7609f4b043eaaf3afcc
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


Keyring is skipped due to an exception: 'keyring.backends'
  Running command git clone --filter=blob:none --quiet https://github.com/CornellNLP/Cornell-Conversational-Analysis-Toolkit.git 'C:\Users\jonat\AppData\Local\Temp\pip-req-build-ffhip93l'


In [3]:
from convokit import Corpus, download

In [5]:
base_subreddit = 'Christianity'

In [6]:
# subreddits = ['exchristian', 'TrueChristian', 'OpenChristian', 'RadicalChristianity', 'Catholicism', 'Judaism', 'Buddhism', 'OrthodoxChristianity', 'Reformed']
# subreddits = ['exchristian']
subreddits = ['TrueChristian', 'OpenChristian']

In [7]:
corpuses = {}

In [9]:
for subreddit in subreddits + [base_subreddit]:
    print('1 Downloading', subreddit)
    corpuses[subreddit] = Corpus.reconnect_to_db(f'subreddit-{subreddit}')
    # if subreddit not in corpuses:
        # corpuses[subreddit] = Corpus(filename=download('subreddit-'+subreddit, use_local=True), storage_type='db')
    print(corpuses[subreddit].print_summary_stats())

1 Downloading TrueChristian
Number of Speakers: 22002
Number of Utterances: 580305
Number of Conversations: 30415
None
1 Downloading OpenChristian
Number of Speakers: 4876
Number of Utterances: 41895
Number of Conversations: 7544
None
1 Downloading Christianity
Number of Speakers: 194228
Number of Utterances: 7327402
Number of Conversations: 325059
None


In [10]:
utts_dfs = {}

In [ ]:
for subreddit in subreddits + [base_subreddit]:
    print('2 Getting dataframes', subreddit)
    corpus = corpuses[subreddit]
    if subreddit not in utts_dfs:
        utts_dfs[subreddit] = corpus.get_utterances_dataframe()

2 Getting dataframes TrueChristian
2 Getting dataframes OpenChristian
2 Getting dataframes Christianity


In [ ]:
for subreddit in subreddits + [base_subreddit]:
    print('3 Calculating lengths', subreddit)
    utts_dfs[subreddit]['text_len'] = utts_dfs[subreddit]['text'].apply(lambda t: len(t.split()))

In [ ]:
for subreddit in subreddits + [base_subreddit]:
    print('4 Removing empty', subreddit)
    df = utts_dfs[subreddit]
    utts_dfs[subreddit] = df[df['text_len'] > 5]
    print(utts_dfs[subreddit].describe())

In [ ]:
speakers = {}

In [ ]:
for subreddit in subreddits + [base_subreddit]:
    print('5 Getting speakers', subreddit)
    speakers[subreddit] = utts_dfs[subreddit]['speaker'].unique()

In [ ]:
common_speakers = {}

In [ ]:
for subreddit in subreddits:
    print('6 Getting common speakers', subreddit)
    speakers = set(speakers[base_subreddit]).intersection(speakers[subreddit])
    speakers.remove('[deleted]')
    common_speakers[subreddit] = list(speakers)
    print(len(speakers))

In [ ]:
common_speakers_utts = {}

In [ ]:
for subreddit in subreddits:
    print('7 Getting utts from common speakers', subreddit)
    df = utts_dfs[base_subreddit]
    common_speakers_utts[subreddit] = df[df['speaker'].isin(common_speakers[subreddit])]

In [ ]:
first_utts = {}

In [ ]:
for subreddit in subreddits + [base_subreddit]:
    print('8 Getting first utt time', subreddit)
    df = utts_dfs[subreddit]
    first_utts[subreddit] = df.groupby(['speaker'])['timestamp'].min()

In [ ]:
for subreddit in subreddits:
    print('9 Getting is before', subreddit)
    df = common_speakers_utts[subreddit]
    df['is_before'] = df.apply(lambda utt: utt['timestamp'] < first_utts[subreddit][utt['speaker']], axis=1)
    common_speakers_utts[subreddit] = df

In [ ]:
import pandas as pd

In [ ]:
for subreddit in subreddits:
    print('10 Ordering', subreddit)
    df = common_speakers_utts[subreddit]
    
    before = df[df['is_before'] == True]
    gb = before.sort_values(['speaker', 'timestamp'], ascending=False).groupby(['speaker'])
    before['order'] = -1 * (gb.cumcount() + 1)
    
    after = df[df['is_before'] == False]
    ga = after.sort_values(['speaker', 'timestamp']).groupby(['speaker'])
    after['order'] = ga.cumcount() + 1
    
    common_speakers_utts[subreddit] = pd.concat([before, after])

In [ ]:
k = 50

In [ ]:
import numpy as np

In [ ]:
import scipy.stats as st
def conf_interval(data):
    interval = st.norm.interval(alpha=0.95, loc=np.mean(data), scale=st.sem(data))
    return (interval[-1] - interval[0])/2

In [ ]:
utts_dfs['exchristian']['text_len'].mean()

In [ ]:
for subreddit in subreddits:
    print('11 Graphing', subreddit)
    len_avg = utts_dfs[subreddit]['text_len'].mean()
    df = common_speakers_utts[subreddit]
    k_only = df[df['order'].abs() <= k]
    g = k_only.groupby(['order']).agg({"text_len": ["mean", conf_interval]})
    g.columns = g.columns.map(lambda x: '_'.join([str(i) for i in x]))
    g = g.reset_index()
    gp = g.plot('order', 'text_len_mean', yerr='text_len_conf_interval')
    gp.axhline(len_avg, color='red', linestyle='--')
    gp.axvline(0, color='grey', linestyle='-')
    
    for i in [5, 10, 20, k/2]:
        g2 = g.groupby(np.arange(len(g))//i).mean()
        g2p = g2.plot('order', 'text_len_mean', yerr='text_len_conf_interval')
        g2p.axhline(len_avg, color='red', linestyle='--')
        g2p.axvline(0, color='grey', linestyle='-')
    